In [21]:
import pandas as pd
import numpy as np
#import git
from datetime import datetime as dt
pd.set_option("display.max_columns", None)

In [ ]:
repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")
df = pd.read_csv(f"{repo}/data/mtd_combined.csv")
df = df[(df["state"] == "completed")]

In [35]:
df = pd.read_csv("../data/mtd_combined.csv", index_col=0)
df['pickup_arrival_time']

0        00:10:37
1             NaN
2        00:12:59
3             NaN
4             NaN
           ...   
10343         NaN
10344         NaN
10345         NaN
10346    00:02:48
10347    00:17:51
Name: pickup_arrival_time, Length: 10348, dtype: object

In [32]:
df["pickup_arrival_time"][1568]

'99:99:99'

In [30]:
pickup_arrival_time = df["pickup_arrival_time"].fillna("99:99:99")
i = 0
for x in pickup_arrival_time:
    if len(x) != 8:
        print(x, i)
    i = i + 1

In [3]:
def clean_created_at(df):
    created_at = pd.to_datetime(df["created_at"])
    return created_at

In [4]:
df["created_at"] = clean_created_at(df)

In [5]:
def clean_scheduled_to(df):
    scheduled_to = pd.to_datetime(df["scheduled_to"])
    scheduled_to = scheduled_to.fillna(df["created_at"])

    scheduled_to = np.where(
        scheduled_to < df["created_at"],
        df["created_at"],
        scheduled_to
    )
    return scheduled_to

In [6]:
df["scheduled_to"] = clean_scheduled_to(df)

In [7]:
def clean_dispatched_at(df):
    # Cast to correct dtype
    dispatched_at = pd.to_datetime(df["dispatched_at"])
    
    # Fill missing values of dispatched_at
    dispatched_at = np.where(
        dispatched_at.isna(),
        df["created_at"],
        dispatched_at
    )
    
    # Check correct ordering
    dispatched_at = np.where(
        (dispatched_at <= df["created_at"]) | (dispatched_at <= df["scheduled_to"] - pd.Timedelta(minutes = 9)),
        df["scheduled_to"] - pd.Timedelta(minutes = 8),
        dispatched_at
    )
    dispatched_at = pd.to_datetime(df["dispatched_at"])
    return dispatched_at

In [9]:
df["dispatched_at"] = clean_dispatched_at(df)

In [10]:
def clean_vehicle_arrived_at(df):
    arriving_push = pd.to_datetime(df["arriving_push"])
    vehicle_arrived_at = pd.to_datetime(df["vehicle_arrived_at"])
    pickup_at = pd.to_datetime(df["pickup_at"])
    
    pickup_arrival_time = (vehicle_arrived_at - df['dispatched_at']).dt.seconds  
    pickup_arrival_time = pickup_arrival_time.fillna(-9)

    
    avg_pickup_arrival_time = sum(x for x in pickup_arrival_time if x != -9) / len(list(x for x in pickup_arrival_time if x != -9))
    print(avg_pickup_arrival_time)


    vehicle_arrived_at = np.where(
        vehicle_arrived_at.isna(),
        np.where(
            pickup_arrival_time == -9,
            np.where(
                (df["dispatched_at"] + pd.Timedelta(seconds=avg_pickup_arrival_time)
                < pickup_at) | (pickup_at.isna() == True),
                df["dispatched_at"] + pd.Timedelta(seconds=avg_pickup_arrival_time),
                pickup_at,
            ),
            np.where(
                (df["dispatched_at"] + pd.to_timedelta(pickup_arrival_time) < pickup_at) | (pickup_at.isna() == True),
                df["dispatched_at"] + pd.to_timedelta(pickup_arrival_time),
                pickup_at,
            ),
        ),
        vehicle_arrived_at,
    )

    vehicle_arrived_at = pd.to_datetime(vehicle_arrived_at)

    # Check ordering
    vehicle_arrived_at = np.where(
        (vehicle_arrived_at < arriving_push) | (vehicle_arrived_at + pd.Timedelta(minutes = 60) < df["scheduled_to"]),
        np.where(
            arriving_push.isna(),
            np.where(
                (df["dispatched_at"] + pd.to_timedelta(pickup_arrival_time) < df["pickup_at"]) | (df["pickup_at"].isna() == True),
                df["dispatched_at"] + pd.Timedelta(seconds=avg_pickup_arrival_time),
                df["pickup_at"]

            ),
            arriving_push
        
        ),
        vehicle_arrived_at
    )

    vehicle_arrived_at = pd.to_datetime(vehicle_arrived_at)
    vehicle_arrived_at = vehicle_arrived_at.floor("s")
    
    return vehicle_arrived_at

In [12]:
df["vehicle_arrived_at"] = clean_vehicle_arrived_at(df)

512.351376574895


In [13]:
def clean_arriving_push(df):
    arriving_push = pd.to_datetime(df["arriving_push"])
    arriving_push = arriving_push.fillna(
        df["vehicle_arrived_at"] - pd.Timedelta(minutes=3)
    )
    return arriving_push

In [14]:
df["arriving_push"] = clean_arriving_push(df)

In [15]:
def clean_pickup_arrival_time(df):
    pickup_arrival_time = (df['vehicle_arrived_at'] - df['dispatched_at']).dt.seconds

    return pickup_arrival_time

In [16]:
df["pickup_arrival_time"] = clean_pickup_arrival_time(df)

In [19]:
def clean_earlierst_pickup_expectation(df):
    earlierst_pickup_expectation = pd.to_datetime(df["earliest_pickup_expectation"])
    earlierst_pickup_expectation = df["dispatched_at"] + pd.Timedelta(minutes = 3)

    return earlierst_pickup_expectation

In [20]:
df["earliest_pickup_expectation"] = clean_earlierst_pickup_expectation(df)

In [31]:
def clean_pick_up_at(df):
    pickup_at = pd.to_datetime(df["pickup_at"])
 
    boarding_time = pd.Series(np.where(
        df["vehicle_arrived_at"] < pickup_at,
        (pickup_at - df["vehicle_arrived_at"]).dt.seconds,
        0
    ))
    boarding_time = boarding_time.fillna(-9)

    avg_boarding_time = sum(x for x in boarding_time if x != -9) / len(list(x for x in boarding_time if x != -9))
    print(avg_boarding_time)
    pickup_at = np.where(
        pickup_at.isna() == True,
        np.where(
            df["pickup_eta"].isna(),
            df['vehicle_arrived_at'] +  pd.Timedelta(avg_boarding_time),
            df["pickup_eta"]
        ),
        pickup_at
    )

    return pickup_at



In [32]:
df["pickup_at"] = clean_pick_up_at(df)

167.08864664747017


In [242]:
def clean_dropoff_at(df):
    
    pass 

0       2021-07-01 07:33:08
2       2021-07-01 09:35:40
6       2021-07-01 17:22:06
7       2021-07-01 18:33:23
8       2021-07-01 18:53:59
                ...        
10300   2021-08-30 22:53:10
10308   2021-08-31 11:21:36
10316   2021-08-31 13:52:50
10327   2021-08-31 17:22:04
10332   2021-08-31 18:37:22
Name: pickup_at, Length: 3832, dtype: datetime64[ns]

In [201]:
def clean_pickup_first_eta(df):
    pass


0

In [ ]:
df["pickup_first_eta"] = clean_pickup_first_eta(df)

In [214]:
df["boarding_time"].str.len()

0        8.0
2        8.0
6        8.0
7        8.0
8        8.0
        ... 
10340    NaN
10341    NaN
10342    NaN
10343    NaN
10345    NaN
Name: boarding_time, Length: 4861, dtype: float64